In [2]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_score, recall_score

# Đọc tệp dữ liệu Cancer.csv trong file data_cancer
df = pd.read_csv('/home/vitquay1708/Study_Space/ml/week2/data/bernoulli_nb_symptoms.csv')

#print(df.head() )


###### Tính các xác suất p_k := N_k / N (chú ý N = len(df) )

In [ ]:
target_feature = 'condition'

Y = df[target_feature]
labels = np.zeros((len(set(Y)), 2), dtype=Y.dtype) 

# Liệt kê các nhãn (label) khác nhau (từ trường cuisine) k =1, 2, ... C
# và tính xác suất P_k của mỗi nhãn. Sử dụng nhãn thay cho chỉ số
id = 0
for label in set(Y):
    labels[id, 0] = label
    labels[id, 1] = (Y == label).sum()/len(Y)
    id += 1

0       food_poison
1               flu
2               flu
3       stomach_bug
4       food_poison
           ...     
2495     covid_like
2496    dehydration
2497    stomach_bug
2498           cold
2499     covid_like
Name: condition, Length: 2500, dtype: object


In [23]:
print(labels)

[['flu' np.float64(0.1532)]
 ['cold' np.float64(0.1576)]
 ['food_poison' np.float64(0.1064)]
 ['allergy' np.float64(0.1184)]
 ['stomach_bug' np.float64(0.1236)]
 ['migraine' np.float64(0.0936)]
 ['covid_like' np.float64(0.1532)]
 ['dehydration' np.float64(0.094)]]


###### Tính các giá trị P_ik ứng với xác suất để trường thứ i nhận giá trị 1: P_ik := [|{xm in class k : x_im = xi}| ] / [N_k]

In [5]:
def Pik_feature_per_class(X):
#Returns the prob. of feature per class
    X = np.array(X)
    # count the number of element x_m wich has field i^th : x_im = 1
    count = (X == 1).sum()
    # P_ik := [|{xm in class k : x_im = 1}|  / N_k 
    return count /len(X)
    

###### Lấy tên mỗi trường (field) của dữ liệu và số lựa chọn ứng với trường đó

In [6]:
def predict_output_label(X, x_input, p_labels, target_feature):

    #num_labels_M = get_distinct_value_in_fields()
    #score for each class 
    p = np.log(np.array(p_labels[:, 1], dtype=float))

    for k in range(len(p)):
        Xk = X[X[target_feature] == p_labels[k, 0]]
        
        for i in range(len(x_input)):
            Pik = Pik_feature_per_class(Xk.iloc[:, i+1])
            if(Pik != 0):
                p[k] += np.log(x_input.iloc[i] * Pik + (1 - x_input.iloc[i]) * (1 - Pik) )

    y_star = np.argmax(p)

    return p_labels[y_star, 0]

In [ ]:
Train_size = round(len(df)*0.6)
X_Train = df[:Train_size]
Y_Train = df[target_feature][:Train_size]

X_Test = df[Train_size:].drop(target_feature, axis = 1).reset_index(drop=True, inplace=False)
Y_Test = df[target_feature][Train_size:].reset_index(drop=True, inplace=False)

Y_pred = np.zeros(len(Y_Test), dtype = Y_Test.dtype) 

for i in range(len(X_Test)):
    Y_pred[i] = predict_output_label(X_Train, X_Test.iloc[i, :], labels, target_feature)

print(Y_pred)

['migraine' 'food_poison' 'cold' 'stomach_bug' 'migraine' 'migraine'
 'cold' 'flu' 'flu' 'stomach_bug' 'stomach_bug' 'flu' 'allergy' 'cold'
 'cold' 'stomach_bug' 'covid_like' 'dehydration' 'cold' 'food_poison'
 'allergy' 'food_poison' 'food_poison' 'allergy' 'stomach_bug' 'migraine'
 'flu' 'covid_like' 'food_poison' 'cold' 'flu' 'cold' 'cold' 'flu'
 'stomach_bug' 'cold' 'stomach_bug' 'food_poison' 'migraine' 'cold' 'cold'
 'allergy' 'covid_like' 'covid_like' 'flu' 'flu' 'dehydration' 'cold'
 'cold' 'allergy' 'dehydration' 'cold' 'stomach_bug' 'covid_like'
 'allergy' 'cold' 'flu' 'food_poison' 'allergy' 'food_poison' 'flu'
 'allergy' 'allergy' 'allergy' 'allergy' 'food_poison' 'flu' 'flu' 'cold'
 'stomach_bug' 'food_poison' 'food_poison' 'flu' 'food_poison' 'flu'
 'allergy' 'migraine' 'cold' 'cold' 'covid_like' 'cold' 'allergy'
 'covid_like' 'food_poison' 'covid_like' 'covid_like' 'covid_like'
 'covid_like' 'migraine' 'dehydration' 'flu' 'migraine' 'food_poison'
 'covid_like' 'allergy' 

In [12]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_pred, Y_Test))

0.479


In [ ]:
print(predict_output_label(X_Train, X_Train.iloc[3, 1:], labels, target_feature) )

stomach_bug


# Sử dụng thư viện scikit learn

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.naive_bayes import BernoulliNB
X_train_lib = X_Train.drop(target_feature, axis = 1).reset_index(drop = True, inplace=False)

encode = LabelEncoder()

for col in X_train_lib.select_dtypes(include=['object', 'category']).columns:
  X_train_lib[col] = encode.fit_transform(X_train_lib[col])

for col in X_Test.select_dtypes(include=['object', 'category']).columns:
  X_Test[col] = encode.fit_transform(X_Test[col])

# print(X_train_lib)
print(len(Y_Train))
clf =  BernoulliNB()
clf.fit(X_train_lib, Y_Train)
y_pred_lib = clf.predict(X_Test)

print(accuracy_score(y_pred_lib, Y_Test))


1500
0.485
